<h2> 3.6 Featurizing text data with tfidf weighted word-vectors </h2>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import time
import warnings
import numpy as np
from nltk.corpus import stopwords
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
warnings.filterwarnings("ignore")
import sys
import os 
import pandas as pd
import numpy as np
from tqdm import tqdm

# exctract word2vec vectors
# https://github.com/explosion/spaCy/issues/1721
# http://landinghub.visualstudio.com/visual-cpp-build-tools
import spacy

In [3]:
# avoid decoding problems
df = pd.read_csv("/content/drive/MyDrive/quora question pair similarity/train.csv")
 
# encode questions to unicode
# https://stackoverflow.com/a/6812069
# ----------------- python 2 ---------------------
# df['question1'] = df['question1'].apply(lambda x: unicode(str(x),"utf-8"))
# df['question2'] = df['question2'].apply(lambda x: unicode(str(x),"utf-8"))
# ----------------- python 3 ---------------------
df['question1'] = df['question1'].apply(lambda x: str(x))
df['question2'] = df['question2'].apply(lambda x: str(x))

In [4]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
# merge texts
questions = list(df['question1']) + list(df['question2'])

tfidf = TfidfVectorizer(lowercase=False, )
tfidf.fit_transform(questions)

# dict key:word and value:tf-idf score
word2tfidf = dict(zip(tfidf.get_feature_names(), tfidf.idf_))

- After we find TF-IDF scores, we convert each question to a weighted average of word2vec vectors by these scores.
- here we use a pre-trained GLOVE model which comes free with "Spacy".  https://spacy.io/usage/vectors-similarity
- It is trained on Wikipedia and therefore, it is stronger in terms of word semantics. 

In [6]:
# en_vectors_web_lg, which includes over 1 million unique vectors.
nlp = spacy.load('en_core_web_sm')

vecs1 = []
# https://github.com/noamraph/tqdm
# tqdm is used to print the progress bar
for qu1 in tqdm(list(df['question1'])):
    doc1 = nlp(qu1) 
    # 384 is the number of dimensions of vectors 
    mean_vec1 = np.zeros([len(doc1), len(doc1[0].vector)])
    for word1 in doc1:
        # word2vec
        vec1 = word1.vector
        # fetch df score
        try:
            idf = word2tfidf[str(word1)]
        except:
            idf = 0
        # compute final vec
        mean_vec1 += vec1 * idf
    mean_vec1 = mean_vec1.mean(axis=0)
    vecs1.append(mean_vec1)
df['q1_feats_m'] = list(vecs1)


100%|██████████| 404290/404290 [56:45<00:00, 118.70it/s]


In [7]:
vecs2 = []
for qu2 in tqdm(list(df['question2'])):
    doc2 = nlp(qu2) 
    mean_vec2 = np.zeros([len(doc1), len(doc2[0].vector)])
    for word2 in doc2:
        # word2vec
        vec2 = word2.vector
        # fetch df score
        try:
            idf = word2tfidf[str(word2)]
        except:
            #print word
            idf = 0
        # compute final vec
        mean_vec2 += vec2 * idf
    mean_vec2 = mean_vec2.mean(axis=0)
    vecs2.append(mean_vec2)
df['q2_feats_m'] = list(vecs2)

100%|██████████| 404290/404290 [55:53<00:00, 120.57it/s]


In [16]:
#prepro_features_train.csv (Simple Preprocessing Feartures)
#nlp_features_train.csv (NLP Features)
if os.path.isfile('/content/drive/MyDrive/quora question pair similarity/nlp_features_train.csv'):
    dfnlp = pd.read_csv("/content/drive/MyDrive/quora question pair similarity/nlp_features_train.csv",encoding='latin-1')
else:
    print("download nlp_features_train.csv from drive or run previous notebook")

if os.path.isfile('/content/drive/MyDrive/quora question pair similarity/df_fe_without_preprocessing_train.csv'):
    dfppro = pd.read_csv("/content/drive/MyDrive/quora question pair similarity/df_fe_without_preprocessing_train.csv",encoding='latin-1')
else:
    print("download df_fe_without_preprocessing_train.csv from drive or run previous notebook")

In [9]:
df1 = dfnlp.drop(['qid1','qid2','question1','question2'],axis=1)
df2 = dfppro.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)
df3 = df.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)
df3_q1 = pd.DataFrame(df3.q1_feats_m.values.tolist(), index= df3.index)
df3_q2 = pd.DataFrame(df3.q2_feats_m.values.tolist(), index= df3.index)

In [10]:
# dataframe of nlp features
df1.head()

,id,is_duplicate,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,token_set_ratio,token_sort_ratio,fuzz_ratio,fuzz_partial_ratio,longest_substr_ratio
0,0,0,0.999980,0.833319,0.999983,0.999983,0.916659,0.785709,0.0,1.0,2.0,13.0,100,93,93,100,0.982759
1,1,0,0.799984,0.399996,0.749981,0.599988,0.699993,0.466664,0.0,1.0,5.0,12.5,86,63,66,75,0.596154
2,2,0,0.399992,0.333328,0.399992,0.249997,0.399996,0.285712,0.0,1.0,4.0,12.0,66,66,54,54,0.166667
3,3,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,2.0,12.0,36,36,35,40,0.039216
4,4,0,0.399992,0.199998,0.999950,0.666644,0.571420,0.307690,0.0,1.0,6.0,10.0,67,47,46,56,0.175000


In [11]:
# data before preprocessing 
df2.head()

,id,freq_qid1,freq_qid2,q1len,q2len,q1_n_words,q2_n_words,word_Common,word_Total,word_share,freq_q1+q2,freq_q1-q2
0,0,1,1,66,57,14,12,10.0,23.0,0.434783,2,0
1,1,4,1,51,88,8,13,4.0,20.0,0.200000,5,3
2,2,1,1,73,59,14,10,4.0,24.0,0.166667,2,0
3,3,1,1,50,65,11,9,0.0,19.0,0.000000,2,0
4,4,3,1,76,39,13,7,2.0,20.0,0.100000,4,2


In [12]:
# Questions 1 tfidf weighted word2vec
df3_q1.head()

,0,1,2,3,4,5,6,7,8,9,...,86,87,88,89,90,91,92,93,94,95
0,28.505168,-6.668543,27.934619,-35.303677,-13.590169,2.813855,-34.141832,-39.723892,13.908162,7.642761,...,21.425739,-1.578822,-26.098868,14.420271,-43.689805,-52.154671,39.668683,65.215845,3.648807,2.141548
1,80.920736,14.522509,8.747215,-10.627093,-32.863597,42.647875,-49.310862,-4.615759,24.387455,-10.313336,...,9.535705,-9.775651,-67.354446,7.883729,7.294851,-46.185484,-18.570708,47.658456,28.385392,26.411261
2,16.304034,10.466194,-16.121922,-2.034769,-12.671675,-3.325117,-4.964587,-40.802416,19.395482,0.458789,...,36.500579,14.436685,5.828354,36.516149,-31.183511,-33.911765,-0.510468,-18.043430,6.461991,-8.763262
3,25.437092,60.496401,-18.734636,-17.648518,10.607407,34.267028,-7.640634,-19.066555,36.765855,-18.292648,...,-17.919563,17.959610,-11.391149,6.164858,1.401957,10.194025,32.522198,-20.278096,36.932516,-12.494729
4,89.659927,-33.571027,32.713714,-32.018297,38.478466,14.766149,-15.869067,-72.610141,86.239486,-5.095635,...,-43.475572,1.631001,-22.070799,35.274364,34.909424,-44.466848,14.255975,29.884435,32.549382,49.327245


In [13]:
# Questions 2 tfidf weighted word2vec
df3_q2.head()

,0,1,2,3,4,5,6,7,8,9,...,86,87,88,89,90,91,92,93,94,95
0,24.072403,-4.327914,16.469961,-28.528379,-8.282737,1.902850,-16.366967,-35.049852,4.878676,1.208403,...,28.148540,1.353424,-11.824829,11.047364,-46.344898,-48.177294,43.835781,34.804440,9.032400,-10.397829
1,70.501072,22.793629,2.456059,7.793115,3.179906,64.958790,-50.533134,-2.921828,32.286931,-11.632442,...,9.847465,-6.033753,-41.089501,14.710465,-2.295958,-41.452774,-8.047283,32.255509,49.435575,7.736098
2,-4.784395,25.616144,32.284418,-17.503802,-28.339841,18.235217,-32.095355,-18.613924,25.132255,38.009239,...,17.234991,-16.020406,11.371594,38.165762,-58.871166,-58.693419,25.354105,-8.443720,18.219843,4.179047
3,8.964527,52.532704,-3.684324,0.089238,-35.289630,10.542750,12.369892,-5.454315,16.958829,9.866997,...,-0.328744,2.499027,13.687267,14.945719,-38.077516,-27.151416,-10.715019,-11.844045,7.744034,-38.738776
4,32.101281,-15.166131,-2.571749,-5.634248,5.786916,20.789494,-6.125809,-17.332545,15.784926,1.939613,...,9.265303,6.862868,-24.873212,18.215863,-8.750956,-22.902639,24.033338,-9.201678,23.656194,0.068456


In [14]:
print("Number of features in nlp dataframe :", df1.shape[1])
print("Number of features in preprocessed dataframe :", df2.shape[1])
print("Number of features in question1 w2v  dataframe :", df3_q1.shape[1])
print("Number of features in question2 w2v  dataframe :", df3_q2.shape[1])
print("Number of features in final dataframe  :", df1.shape[1]+df2.shape[1]+df3_q1.shape[1]+df3_q2.shape[1])

Number of features in nlp dataframe : 17
Number of features in preprocessed dataframe : 12
Number of features in question1 w2v  dataframe : 96
Number of features in question2 w2v  dataframe : 96
Number of features in final dataframe  : 221


In [15]:
# storing the final features to csv file
if not os.path.isfile('/content/drive/MyDrive/quora question pair similarity/final_features.csv'):
    df3_q1['id']=df1['id']
    df3_q2['id']=df1['id']
    df1  = df1.merge(df2, on='id',how='left')
    df2  = df3_q1.merge(df3_q2, on='id',how='left')
    result  = df1.merge(df2, on='id',how='left')
    result.to_csv('/content/drive/MyDrive/quora question pair similarity/final_features.csv')